In [1]:
cd ..

C:\Users\d_mart04\Google Drive\Doutorado\gitworkspace\phd-query-synthesis\PhDCoding


In [2]:
import sys, os
sys.path.append('..\\PhDCoding\\qbe')

In [3]:
import random
import numpy as np, pandas as pd, pandasql as pdsql, sklearn.datasets as ds
import util
from qbe.greedyqbe import GreedySearchQBE
from qbe.fitfunction import QueryDiscoveryFitnessFunction
from qbe.deapgpqbe import DEAPGeneticProgrammingQBE

In [4]:
def get_view(predicate):
    query = "SELECT * FROM dataframe WHERE " + predicate
    print('Query: ', query)
    print('---------------------------------')
    best_data_view = pysql(query)
    util.get_information_retrieval_metrics(dataframe, desired_output, best_data_view)
    print('---------------------------------')
    return best_data_view

def get_desired_indexes(dataframe, desired_output):
    indexes = []
    index = 0
    for row in dataframe.values:
        for out in desired_output.values: 
            if (row == out).all():
                indexes.append(index)
        index += 1
    return indexes

#### Loading the Iris dataset https://archive.ics.uci.edu/ml/datasets/iris

In [5]:
iris = ds.load_iris()
dataframe = pd.DataFrame(data= np.c_[iris['data'], iris['target']], columns= iris['feature_names'] + ['target'])
dataframe.columns = ["sepal_len", "sepal_wid", "petal_len", "petal_wid", "target"]
dataframe

,sepal_len,sepal_wid,petal_len,petal_wid,target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
5,5.4,3.9,1.7,0.4,0.0
6,4.6,3.4,1.4,0.3,0.0
7,5.0,3.4,1.5,0.2,0.0
8,4.4,2.9,1.4,0.2,0.0
9,4.9,3.1,1.5,0.1,0.0


#### Configure PandaSQL to query Pandas dataframe

In [6]:
pysql = lambda q: pdsql.sqldf(q, globals())

#### Create the view for the query reverse engineering problem

In [8]:
desired_output = pysql("SELECT * FROM dataframe WHERE petal_len >= 4.0 AND petal_wid = 1.8 AND target = 1")
desired_indices = get_desired_indexes(dataframe, desired_output)
#size_desired_set = random.randint(0, 20)
#desired_indexes = [random.randint(0, dataframe.shape[0]-1) for x in range(size_desired_set)]
desired_output = dataframe.loc[desired_indices]
desired_output

,sepal_len,sepal_wid,petal_len,petal_wid,target
70,5.9,3.2,4.8,1.8,1.0


#### Fitness Function

In [9]:
fitness_function = QueryDiscoveryFitnessFunction(dataframe, desired_indices)

#### Greedy Search

In [10]:
greedy_search = GreedySearchQBE(fitness_function, dataframe)

In [14]:
%time best_greedy = greedy_search.search_best_predicate(max_iterations=100, threshold=0.001)

------------------------------
Iteration 	 Best fitness
1 		 1.3423
2 		 0.0000
-------- Search ended --------
Wall time: 338 ms


In [16]:
get_view(best_greedy)

Query:  SELECT * FROM dataframe WHERE (sepal_len == 5.9) AND (sepal_wid > 3.0)
---------------------------------
Relevant & Actual:  1
Recall:  1.0
Specificity:  1.0
Precision:  1.0
F1-Score:  1.0
False negative rate:  0.0
False positive rate:  0.0
---------------------------------


,sepal_len,sepal_wid,petal_len,petal_wid,target
0,5.9,3.2,4.8,1.8,1.0


#### Decision Tree

In [17]:
from treeqbe import DecisionTreeQBE

In [19]:
decision_tree = DecisionTreeQBE(dataframe, desired_indices)

In [22]:
%time best_tree = decision_tree.search_best_predicate()

Wall time: 991 µs


In [23]:
get_view(best_tree)

Query:  SELECT * FROM dataframe WHERE petal_wid > 1.75 AND target <= 1.5
---------------------------------
Relevant & Actual:  1
Recall:  1.0
Specificity:  1.0
Precision:  1.0
F1-Score:  1.0
False negative rate:  0.0
False positive rate:  0.0
---------------------------------


,sepal_len,sepal_wid,petal_len,petal_wid,target
0,5.9,3.2,4.8,1.8,1.0


#### Genetic Programming

In [24]:
genetic_programming = DEAPGeneticProgrammingQBE(dataframe, fitness_function)

In [27]:
%time best_genetic = genetic_programming.search_best_predicate(population_size=200, crossover_rate=0.9, mutation_rate=0.5, num_generations=100, max_gen_without_gain=30, verbose=False)

Wall time: 6.49 s


In [28]:
get_view(best_genetic)

Query:  SELECT * FROM dataframe WHERE ((petal_len >= 4.8) AND (petal_len <= 4.8))
---------------------------------
Relevant & Actual:  1
Recall:  1.0
Specificity:  0.9797297297297297
Precision:  0.25
F1-Score:  0.4
False negative rate:  0.0
False positive rate:  0.020270270270270285
---------------------------------


,sepal_len,sepal_wid,petal_len,petal_wid,target
0,5.9,3.2,4.8,1.8,1.0
1,6.8,2.8,4.8,1.4,1.0
2,6.2,2.8,4.8,1.8,2.0
3,6.0,3.0,4.8,1.8,2.0
